In [2]:

# Set up
import requests
from collections import OrderedDict 
from operator import itemgetter
import urllib
from bs4 import BeautifulSoup
from bs4.element import Comment
from collections import Counter
from string import punctuation
import re
from multiprocessing import Pool
import multiprocessing
from collections import deque
import math

In [6]:
WORDS = {}
visited = set()

def make_soup(url):
    try:
        page = urllib.request.urlopen(url)
    except urllib.request.HTTPError as e:
        print('Ignored: ', e)
        return -1
        
    req = urllib.request.Request(url , headers={'User-Agent': 'Mozilla/5.0'})
    page = urllib.request.urlopen(req).read()
    soup_data = BeautifulSoup(page, "html.parser")
    return soup_data


def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True


def text_from_html(url):
    soup = make_soup(url)
    if soup == -1:
        return ''
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)


def get_words(url):
    text = text_from_html(url)
    # Delete punctuaction and empty words
    text = [re.sub('\W+', '', t) for t in text.lower().split()]
    text = list(filter(lambda t: len(t), text))
    return text

def get_hrefs(url):
    soup = make_soup(url)
    if soup == -1:
        return []
    hrefs = []
        
    for link in soup.find_all('a'):
        hrefs.append(link.get('href'))
        
    ans = []
    for link in hrefs:
        if link is None or len(link) == 0:
            continue
        if link.find('https') != -1 or link.find('http') != -1:
            ans.append(link)

          
    return ans

def get_dict():
    return dict(sorted(WORDS.items(), key=lambda x: len(x[1]), reverse=True))

def BFS(url, max_steps):
    Q = deque()
    Q.append((url, 0))
    visited = set()
    visited.add(url)
    
    while len(Q):
        act_url = Q[0][0]
        act_steps = Q[0][1]
        Q.popleft()
    
        print(f'Remaining: {len(Q)}. Act: ', act_url)
        
        words = get_words(act_url)
        hrefs = get_hrefs(act_url)
    
        for w in words:
            WORDS.setdefault(w, set()).add(act_url)

        for no, ref in enumerate(hrefs):
            if act_steps < max_steps and not ref in visited:
                Q.append((ref, act_steps + 1))
                visited.add(ref)
                
                
def scrap(url, depth):
    WORDS.clear()
    BFS(url, depth)
    print(WORDS[0])
    return get_dict()




words = scrap('https://en.wikipedia.org/wiki/Joy_Division', 1)

Remaining: 0. Act:  https://en.wikipedia.org/wiki/Joy_Division
Remaining: 125. Act:  http://joydivisionofficial.com
Remaining: 124. Act:  https://www.miaminewtimes.com/music/five-best-goth-rock-bands-of-all-time-7572335
Remaining: 123. Act:  https://www.nme.com/blogs/nme-blogs/release-the-bats-its-the-20-greatest-goth-tracks-758985
Remaining: 122. Act:  http://www.allmusic.com/artist/new-order-mn0000334193/biography
Remaining: 121. Act:  https://web.archive.org/web/20120104025745/http://www.musicweek.com/story.asp?storyCode=1031301&sectioncode=2


KeyboardInterrupt: 